vm-asm_v1.3 修正点<br>
・フォルダパスを直書きから標準入力に変更

vm-asm_v1.2 修正点<br>
・ParserやCodeWriteが「全部読んで全部吐き出す」動作をしていたため、仕様に忠実な「一行ずつ読んで吐き出す」動作を行うよう修正

vm-asm_v1.1 修正点<br>
・フォルダやそれに含まれる複数ファイルを渡されても動作するよう修正

In [18]:
from google.colab import drive
import os
import glob

In [19]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
def hasMoreCommands(line):
  if line=='/':
    return False
  elif line=='\n':
    return False
  else:
    return True

In [21]:
def commandType(com):
  if com[0]=='p':
    if com[1]=='u':
      return 'C_PUSH'
    elif com[1]=='o':
      return 'C_POP'
  elif com[0]=='l':
    if com[1]=='a':
      return 'C_LABEL'
    else:
      return 'C_ARITHMETIC'
  elif com[0]=='g':
    if com[1]=='o':
      return 'C_GOTO'
    else:
      return 'C_ARITHMETIC'
  elif com[0]=='I':
    return 'C_IF'
  elif com[0]=='F':
    return 'C_FUNCTION'
  elif com[0]=='R':
    return 'C_RETURN'
  elif com[0]=='C':
    return 'C_CALL'
  else:
    return 'C_ARITHMETIC'

In [22]:
def arg1(com,comType):
  if comType=='C_ARITHMETIC':
    spcom=com.split('\n')
    return spcom[0]
  else:
    spcom=com.split(' ')
    return spcom[1]

In [23]:
def arg2(com,comType):
  spcom=com.split(' ')
  return int(spcom[2])

In [24]:
def Parser(line):
  vm=[]
  com=None
  comType=None
  a1=None
  a2=None
  if hasMoreCommands(line[0]):
    com=line
    comType=commandType(com)
    if comType!='C_RETURN':
      a1=arg1(com,comType)
      if comType=='C_POP':
        a2=arg2(com,comType)
      elif comType=='C_PUSH':
        a2=arg2(com,comType)
      elif  comType=='C_FUNCTION':
        a2=arg2(com,comType)
      elif comType=='C_CALL':
        a2=arg2(com,comType)
      else:
        a2=None
    else:
      a1=None
      a2=None
    vm.extend([comType,a1,a2])
    return vm

In [25]:
def writeArithmetic(line,ct):
  hack=[]
  if line[1]=='add':
    tmp='@SP','AM=M-1','D=M','A=A-1','M=D+M'
    hack.extend(tmp)
  elif line[1]=='sub':
    tmp='@SP','AM=M-1','D=M','A=A-1','M=M-D'
    hack.extend(tmp)
  elif line[1]=='neg':
    tmp='@SP','A=M-1','M=-M'
    hack.extend(tmp)
  elif line[1]=='eq':
    ct[0] += 1
    true_l = 'TRUE_EQ' + str(ct[0])
    end_l = 'END_EQ' + str(ct[0])
    tmp='@SP','AM=M-1','D=M','A=A-1','D=M-D','@'+true_l,'D;JEQ','@SP','A=M-1','M=0','@'+end_l,'0;JMP','('+true_l+')','@SP','A=M-1','M=-1','('+end_l+')'
    hack.extend(tmp)
  elif line[1]=='gt':
    ct[1] += 1
    true_l = 'TRUE_GT' + str(ct[1])
    end_l = 'END_GT' + str(ct[1])
    tmp='@SP','AM=M-1','D=M','A=A-1','D=M-D','@'+true_l,'D;JGT','@SP','A=M-1','M=0','@'+end_l,'0;JMP','('+true_l+')','@SP','A=M-1','M=-1','('+end_l+')'
    hack.extend(tmp)
  elif line[1]=='lt':
    ct[2] += 1
    true_l = 'TRUE_LT' + str(ct[2])
    end_l = 'END_LT' + str(ct[2])
    tmp='@SP','AM=M-1','D=M','A=A-1','D=M-D','@'+true_l,'D;JLT','@SP','A=M-1','M=0','@'+end_l,'0;JMP','('+true_l+')','@SP','A=M-1','M=-1','('+end_l+')'
    hack.extend(tmp)
  elif line[1]=='and':
    tmp='@SP','AM=M-1','D=M','A=A-1','M=D&M'
    hack.extend(tmp)
  elif line[1]=='or':
    tmp='@SP','AM=M-1','D=M','A=A-1','M=D|M'
    hack.extend(tmp)
  elif line[1]=='not':
    tmp='@SP','A=M-1','M=!M'
    hack.extend(tmp)
  return hack,ct

In [26]:
def writePushPop(line,filename):
  hack=[]
  if line[0]=='C_PUSH':
    if line[1]=='local':
      tmp='@LCL','D=M','@'+str(line[2]),'A=D+A','D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='argument':
      tmp='@ARG','D=M','@'+str(line[2]),'A=D+A','D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='this':
      tmp='@THIS','D=M','@'+str(line[2]),'A=D+A','D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='that':
      tmp='@THAT','D=M','@'+str(line[2]),'A=D+A','D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='pointer':
      tmp='@'+str(3+int(line[2])),'D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='temp':
      tmp='@'+str(5+int(line[2])),'D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='constant':
      tmp='@'+str(line[2]),'D=A','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    elif line[1]=='static':
      tmp='@'+str(filename)+str(line[2]),'D=M','@SP','A=M','M=D','@SP','M=M+1'
      hack.extend(tmp)
    return hack
  elif line[0]=='C_POP':
    if line[1]=='local':
      tmp='@LCL','D=M','@'+str(line[2]),'D=D+A','@R13','M=D','@SP','M=M-1','A=M','D=M','@R13','A=M','M=D'
      hack.extend(tmp)
    elif line[1]=='argument':
      tmp='@ARG','D=M','@'+str(line[2]),'D=D+A','@R13','M=D','@SP','M=M-1','A=M','D=M','@R13','A=M','M=D'
      hack.extend(tmp)
    elif line[1]=='this':
      tmp='@THIS','D=M','@'+str(line[2]),'D=D+A','@R13','M=D','@SP','M=M-1','A=M','D=M','@R13','A=M','M=D'
      hack.extend(tmp)
    elif line[1]=='that':
      tmp='@THAT','D=M','@'+str(line[2]),'D=D+A','@R13','M=D','@SP','M=M-1','A=M','D=M','@R13','A=M','M=D'
      hack.extend(tmp)
    elif line[1]=='pointer':
      tmp='@'+str(3+int(line[2])),'D=A','@R13','M=D','@SP','M=M-1','A=M','D=M','@R13','A=M','M=D'
      hack.extend(tmp)
    elif line[1]=='temp':
      tmp='@SP','M=M-1','A=M','D=M','@'+str(5+int(line[2])),'M=D'
      hack.extend(tmp)
    elif line[1]=='static':
      tmp='@SP','M=M-1','A=M','D=M','@'+str(filename)+str(line[2]),'M=D'
      hack.extend(tmp)
    return hack

In [27]:
from types import CodeType
def CodeWriter(line,outfile_path,ct):
  hack=[]
  outfile = open(outfile_path, 'a')
  if line[0]=='C_ARITHMETIC':
    tmp,ct=writeArithmetic(line,ct)
    hack.extend(tmp)
  if line[0]=='C_PUSH'or line[0]=='C_POP':
    hack.extend(writePushPop(line,filename))
  for i in hack:
    outfile.write(i+'\n')
  outfile.close()
  return hack,ct

フォルダパスの例 /content/gdrive/MyDrive/3_PBL/07/exp

In [34]:
folder_path = input('フォルダのパスを入力してください\n')
file_extension = '*.vm'
file_list = glob.glob(os.path.join(folder_path, file_extension))
for file in file_list:
    filename = (file.split('/')[-1]).split('.')[0]
    outfile_path=f'{folder_path}/{filename}.asm'
    outfile = open(outfile_path, 'w')
    outfile.truncate(0)
    outfile.close()
    ct=[0,0,0]
    print(filename)
    with open(file) as f:
      for line in f:
        Pline=Parser(line)
        print(Pline)
        if Pline!=None:
          Cfile,ct=CodeWriter(Pline,outfile_path,ct)
          print(Cfile)
    outfile = open(outfile_path, 'a')
    outfile.write('(END)\n@END\n0;JMP')
    outfile.close()

フォルダのパスを入力してください
/content/gdrive/MyDrive/3_PBL/07/exp
PointerTest
None
None
None
None
None
None
None
None
['C_PUSH', 'constant', 3030]
['@3030', 'D=A', '@SP', 'A=M', 'M=D', '@SP', 'M=M+1']
['C_POP', 'pointer', 0]
['@3', 'D=A', '@R13', 'M=D', '@SP', 'M=M-1', 'A=M', 'D=M', '@R13', 'A=M', 'M=D']
['C_PUSH', 'constant', 3040]
['@3040', 'D=A', '@SP', 'A=M', 'M=D', '@SP', 'M=M+1']
['C_POP', 'pointer', 1]
['@4', 'D=A', '@R13', 'M=D', '@SP', 'M=M-1', 'A=M', 'D=M', '@R13', 'A=M', 'M=D']
['C_PUSH', 'constant', 32]
['@32', 'D=A', '@SP', 'A=M', 'M=D', '@SP', 'M=M+1']
['C_POP', 'this', 2]
['@THIS', 'D=M', '@2', 'D=D+A', '@R13', 'M=D', '@SP', 'M=M-1', 'A=M', 'D=M', '@R13', 'A=M', 'M=D']
['C_PUSH', 'constant', 46]
['@46', 'D=A', '@SP', 'A=M', 'M=D', '@SP', 'M=M+1']
['C_POP', 'that', 6]
['@THAT', 'D=M', '@6', 'D=D+A', '@R13', 'M=D', '@SP', 'M=M-1', 'A=M', 'D=M', '@R13', 'A=M', 'M=D']
['C_PUSH', 'pointer', 0]
['@3', 'D=M', '@SP', 'A=M', 'M=D', '@SP', 'M=M+1']
['C_PUSH', 'pointer', 1]
['@4', 'D=M', '@SP'